<a href="https://colab.research.google.com/github/jh6695/study/blob/main/%EC%98%81%ED%99%94_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

캐글 영화 추천 시스템


In [1]:
%matplotlib inline
#notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해줌
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
!pip install surprise
from surprise import Reader, Dataset, SVD
# evaluate가 cross_valiate로 대체
from surprise.model_selection import cross_validate


import warnings; warnings.simplefilter('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163000 sha256=e41c9b208eeb8a08c7b56080c6accc8485bae762f437413b3ced3cd6980e3ecc
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
md = pd.read_csv('/content/drive/MyDrive/the-movies-dataset/movies_metadata.csv')

In [3]:
md.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
md['genres'].isna().sum()

0

In [5]:
#literal_eval - 문자열 리스트로 변환
#isinstance(x, list) - x가 리스트인지
### 'genres' 열을 리스트로 바꾸고 'name'만 추출
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

IMBD 수학 공식을 사용해 평가한다.


> wight rating(wr) = (v / (v + m) * R) + (m / (v + m) * C)


v: 영화의 vote 수

m: 최소한의 vote 수

R: 영화 평점의 평균

C: 영화 평점의 전체 평균



---


최소한의 vote 수를 백분위 95%로 설청한다.

In [6]:
md.shape

(45466, 24)

In [7]:
print(md['vote_average'].isna().sum())
print(md['vote_count'].isna().sum())

6
6


In [8]:
vote_average = md[md['vote_average'].notnull()]['vote_average'].astype('int')
vote_count = md[md['vote_count'].notnull()]['vote_count'].astype('int')

In [9]:
m = vote_count.quantile(0.95)
C = vote_average.mean()

In [10]:
m

434.0

In [11]:
md['release_date'].isna().sum()

87

In [12]:
# year 열 생성
# errors = 'coerce' - 오류 날 경우 NaT로 출력
md['year'] = pd.to_datetime(md['release_date'], errors= 'coerce').apply(lambda x : str(x).split('-')[0] if not(pd.isnull(x)) else np.nan)

In [13]:
# 투표수 95% 열 추출
top = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]

In [14]:
top['vote_average'] = top['vote_average'].astype('int')
top['vote_count']  =top['vote_count'].astype('int')

In [15]:
top.shape

(2274, 6)

In [16]:
def wr(x):
  v = x['vote_count']
  R = x['vote_average']
  return (v/(v+m)*R + (m/(m+v)*C))

In [17]:
top['wr'] = top.apply(wr, axis = 1)

In [18]:
top = top.sort_values('wr', ascending=False)

In [19]:
top

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
...,...,...,...,...,...,...,...
1491,Batman & Robin,1997,1447,4,17.038824,"[Action, Crime, Fantasy]",4.287233
26563,Fantastic Four,2015,2322,4,17.246484,"[Action, Adventure, Science Fiction]",4.196040
30556,Independence Day: Resurgence,2016,2550,4,16.993841,"[Action, Adventure, Science Fiction]",4.181061
21238,Sharknado,2013,484,3,4.928195,"[TV Movie, Horror]",4.061313


상위 3개의 영화가 모두 크리스토퍼 놀란 감독의 영화이다. 관객들은 특정 감독이나 장르에 편향이 존재함을 알 수 있다.

이와 관련된 함수를 만들기 위해 백분위수 95%에서 85%로 완화한다.

In [20]:
#여러개의 장르를 펼쳐 데이터 생성
#stack 들어온 것부터 쌓음
#reset_index - 인덱스를 열로 변환
## level - multi index일 경우 제거할 인덱스 레벨 설정
## drop - 제거한 인덱스를 열로 추가할지 여부
s = md.apply(lambda x: pd.Series(x['genres']), axis = 1).stack().reset_index(level = 1, drop =True)
s.name = 'genre'
gen_md = md.drop('genres', axis = 1).join(s)

In [21]:
gen_md

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Fantasy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Action
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Drama
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,2003,Thriller
45464,False,NaN,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,...,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,1917,NaN


In [22]:
def gc(genre, percentile = 0.85):
  df = gen_md[gen_md['genre'] == genre]

  vote_count = df[df['vote_count'].notnull()]['vote_count'].astype('int')
  vote_average = df[df['vote_average'].notnull()]['vote_average'].astype('int')
  m = vote_count.quantile(percentile)
  C = vote_average.mean()

  top = df[(df['vote_count'] >= m) & (df['vote_average'].notnull())
        & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]

  top['vote_average'] = top['vote_average'].astype('int')
  top['vote_count'] = top['vote_average'].astype('int')

  top['wr'] = top.apply(lambda x: (x['vote_count']/(x['vote_count']+m)*x['vote_average']) + (m/(m+x['vote_count']) * C), axis = 1)
  top = top.sort_values('wr', ascending=False).head(250)

  return top


In [23]:
# 로맨스 영화 top15
gc('Romance')

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,9,9,34.457024,5.728107
45437,In a Heartbeat,2017,8,8,20.82178,5.614135
40251,Your Name.,2016,8,8,34.461252,5.614135
16694,Castaway on the Moon,2009,8,8,6.266701,5.614135
1132,Cinema Paradiso,1988,8,8,14.177005,5.614135
...,...,...,...,...,...,...
36038,My King,2015,7,7,5.88082,5.518263
32342,Battle for Sevastopol,2015,7,7,5.603565,5.518263
1006,The Sound of Music,1965,7,7,9.06773,5.518263
35535,The Last: Naruto the Movie,2014,7,7,5.294086,5.518263


In [24]:
links_small = pd.read_csv('/content/drive/MyDrive/the-movies-dataset/links_small.csv')

In [25]:
links_small

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


Content Based Recommender

위에 개발한 모형은 사용자의 선호와는 무관한 단순 장르별 추천 모형이다.

이를 사용할 시 자신의 선호도와는 무관한 영화를 추천받기 때문에 특정한 측정을 기반으로 영화간 유사성을 계산하는 모형을 개발할 것이다.

- 영화의 개요와 짧은 요약(Overview and Tagline)
- 영화의 캐스팅, 출연자, 키워드, 장르(Cast, Crew, Keywords and Genre)

meta data를 사용해 두 가지 정보에 기반하는 2개의 모델을 만들 것이다.

(컴퓨팅 성능 문제로 모든 영화 데이터는 사용하지 못함.)


In [26]:
links_small = pd.read_csv('/content/drive/MyDrive/the-movies-dataset/links_small.csv')
links_small

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


In [27]:
links_small.isnull().sum()

movieId     0
imdbId      0
tmdbId     13
dtype: int64

In [28]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [29]:
#데이터 축소
#id 열에 0000-00-00 형식의 데이터가 있는 것 확인
md['id'] = md['id'].astype('int')

ValueError: invalid literal for int() with base 10: '1997-08-20'

In [30]:
md = md[md['id'].str.contains('-') == False]
md['id'] = md['id'].astype('int')

In [31]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [32]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['tagline'] + smd['overview']
smd['description'] = smd['description'].fillna('')

In [33]:
tf = TfidfVectorizer(analyzer= 'word', ngram_range=(1,2), min_df=0, stop_words='english')
tfid_matrix = tf.fit_transform(smd['description'])

In [34]:
tfid_matrix.shape

(9099, 269262)

# Cosin Similarity

두 영화 사이의 유사도를 계산하기 위해 코사인 유사도를 사용한다.

$ cosine(x, y) = \frac{x \cdot y^\top}{\parallel x \parallel \cdot \parallel y \parallel} $  

우리가 TF_IDF Vectorizer를 사용했기 때문에, 내적이 바로 Cosine Similarity Score를 제공해준다. 그러므로 우리는 cosine_similarities대신에 더 빠른 sklearn의 linear_kernel을 사용할 것이다.

In [35]:
cosine_sim = linear_kernel(tfid_matrix, tfid_matrix)

In [36]:
cosine_sim[0]

array([1.        , 0.00680204, 0.        , ..., 0.        , 0.00344826,
       0.        ])

In [37]:
#코사인 유사도를 기반으로 가장 유사한 30개의 영화 추천하는 모델
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [38]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [39]:
# enumerate - 인덱스와 값을 포함해서 리턴
# sorted - key: 무엇을 기준으로 정렬할 것인지
def get_recommendations(title):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse =True)
  sim_scores = sim_scores[1:31]

  movie_indices = [i[0] for i in sim_scores]
  return titles.iloc[movie_indices]

In [40]:
get_recommendations('The Dark Knight')

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
6144                              Batman Begins
7933         Sherlock Holmes: A Game of Shadows
5511                            To End All Wars
4489                                      Q & A
7344                        Law Abiding Citizen
7242                  The File on Thelma Jordon
3537                               Criminal Law
2893                              Flying Tigers
1135                   Night Falls on Manhattan
8680                          The Young Savages
8917         Batman v Superman: Dawn of 

다크 나이트를 좋아하는 사람은 베트맨 영화를 많이 추천한다.

그러나 이것은 감독이나 출연진, 장르와 관련된 데이터는 포함하지 않음.

이를 포함한 새로운 모델을 개발한다.


# Metadata Based Recommender

Metadata 기반 모델을 만들기 위해 기존  데이터에 crew 데이터와 keyword 데이터를 합친다.

In [44]:
credits = pd.read_csv('/content/drive/MyDrive/the-movies-dataset/credits.csv')
keywords = pd.read_csv('/content/drive/MyDrive/the-movies-dataset/keywords.csv')

In [42]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [45]:
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [47]:
credits['id'] = credits['id'].astype('int')
keywords['id'] = keywords['id'].astype('int')
md['id'] = md['id'].astype('int')

In [48]:
md.shape

(45463, 25)

In [51]:
md = md.merge(credits, on = 'id')
md = md.merge(keywords, on = 'id')

In [52]:
links_small

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int64

In [57]:
smd = md[md['id'].isin(links_small)]

In [59]:
smd.shape

(9219, 28)

이제 하나의 데이터프레임 안에 감독, 출연진, 장르, credit 정보가 들어 있다. 이를 조금 더 사용 가능하게 조절한다.

crew: crew에서 다른 것들은 영화의 feel에 영향을 끼치지 못하므로 감독만 사용한다.

cast: 덜 유명한 배우와 마이너한 역할은 영항을 크게 미치지 못한다. 임의로 top 3의 배우만 추출한다.

In [62]:
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [64]:
smd['crew']= smd['crew'].apply(literal_eval)
smd['cast']= smd['cast'].apply(literal_eval)
smd['keywords']= smd['keywords'].apply(literal_eval)

In [67]:
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [78]:
smd['crew'][0][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [74]:
def get_director(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
  return np.nan

In [79]:
smd['director'] = smd['crew'].apply(get_director)

In [80]:
smd['cast'][0][0]

{'cast_id': 14,
 'character': 'Woody (voice)',
 'credit_id': '52fe4284c3a36847f8024f95',
 'gender': 2,
 'id': 31,
 'name': 'Tom Hanks',
 'order': 0,
 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}

In [86]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

TypeError: string indices must be integers

In [87]:
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [89]:
smd['keywords'][3][0]

{'id': 818, 'name': 'based on novel'}

In [90]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x , list) else [])

장르, 감독, 주연 및 키워드로 구성된 모든 영화에 대한 데이터 덤프를 만든다. Description Recomment에서 했던 것처럼 **Count Vectorizer**을 사용하여 우리의 count matrix를 개발할 것이다. 이후 cosine similarity를 계산하고 가장 유사한 영화를 반환한다.

1. 모든 요소에서 **공백을 제거하고 소문자로 변환**한다. 이렇게 하면, 우리의 엔진은 Johnny Depp과 Johnny Galecki를 혼동하지 않는다.
2. **감독을 3번 언급**하여 전체 출연진에 비해 더 가중치를 부여한다.

In [94]:
smd['director']= smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director']= smd['director'].apply(lambda x: [x, x, x])

0            [John Lasseter, John Lasseter, John Lasseter]
1               [Joe Johnston, Joe Johnston, Joe Johnston]
2            [Howard Deutch, Howard Deutch, Howard Deutch]
3        [Forest Whitaker, Forest Whitaker, Forest Whit...
4            [Charles Shyer, Charles Shyer, Charles Shyer]
                               ...                        
40952     [Gregg Champion, Gregg Champion, Gregg Champion]
41172    [Tinu Suresh Desai, Tinu Suresh Desai, Tinu Su...
41225    [Ashutosh Gowariker, Ashutosh Gowariker, Ashut...
41391           [Hideaki Anno, Hideaki Anno, Hideaki Anno]
41669                 [Ron Howard, Ron Howard, Ron Howard]
Name: director, Length: 9219, dtype: object

## keywords
keyword의 빈도를 계산한다.

In [96]:
smd['keywords']

0        [jealousy, toy, boy, friendship, friends, riva...
1        [board game, disappearance, based on children'...
2        [fishing, best friend, duringcreditsstinger, o...
3        [based on novel, interracial relationship, sin...
4        [baby, midlife crisis, confidence, aging, daug...
                               ...                        
40952                           [friendship, brick making]
41172                                          [bollywood]
41225                                          [bollywood]
41391    [monster, godzilla, giant monster, destruction...
41669                                 [music, documentary]
Name: keywords, Length: 9219, dtype: object

In [103]:
s = smd.apply(lambda x: pd.Series(x['keywords']), axis = 1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [108]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

1에서 610번의 빈도가 존재한다. 1번 반 발생한 키워드는 사용하지 않겠다.

또한 dog와 dogs 같은 키워드를 같은 키워드로 간주하도록 모든 단어를 어간으로 바꾼다. -- SnowballStemmer

In [116]:
s = s[s>1]

In [111]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [119]:
def filter_keywords(x) :
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [117]:
def filter_keywords(x):
  words = []
  for i in x:
    if i in s:
     words.append(i)
  return words

In [125]:
smd['keywords']= smd['keywords'].apply(filter_keywords)
smd['keywords']= smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords']= smd['keywords'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])

In [130]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))
smd['soup'][0]

'jealousi toy boy friendship friend rivalri boynextdoor newtoy toycomestolif Tom Hanks Tim Allen Don Rickles johnlasseter johnlasseter johnlasseter Animation Comedy Family'

In [132]:
count = CountVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [134]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [135]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index = smd['title'])

우리는 이전에 사용한 get_recommendations 함수를 다시 사용할 것이다. cosind_sim이 업데이트 되었으니 더 나은 성능을 보일 것이다.

In [136]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
1134                Batman Returns
7659    Batman: Under the Red Hood
Name: title, dtype: object

새로운 결과는 조금 더 만족스럽다. 감독 요소에 가중치를 부과했기 때문에 크리스토퍼 놀란 감독의 영화를 상위권에 추천하한다.

이 모델에서는 rating과 popularity를 고려하지 못한다. 따라서 나쁜 영화를 제거하고 좋은 평을 받은 영화를 추천하는 메커니즘을 추가할 것이다.

유사도 점수와 영화의 60%의 vote를 계산하여 Top25 영화를 가져올 것이다. 그런 다음 이것을 M의 값으로 사용하여 Simple Recommender 섹션에서 했던 것처럼 IMDB의 공식을 사용하여 각 영화의 가중치 등급을 계산한다.

In [138]:
idx = indices['The Dark Knight']
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse= True)
sim_scores = sim_scores[1:26]
movie_indices = [i[0] for i in sim_scores]

In [141]:
def improved_recommendations(title):
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.6)
    qualified = movies[(movies['vote_count']>=m)&(movies['vote_count'].notnull())&
                (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(wr, axis = 1)
    qualified = qualified.sort_values('wr', ascending = False).head(10)

    return qualified

In [142]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
524,Batman,2145,7,1989,6.704647
6622,Children of Men,2120,7,2006,6.701756
8419,Man of Steel,6462,6,2013,5.952478
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943


# Collaborative Filtering

이전까지 진행된 content based engine은 특정 영화에 기반된 추천을 한다. 즉, 이는 개인적인 취향은 고려되지 않는다. 이 모델을 사용하는 사람은 모두 같은 영화를 추천받는다.

그러므로, 이번 섹션에서 우리는 **Collaborative Filtering**이라고 불리는 기술을 사용하여 영화 관람객들에게 추천을 한다. Collaborative Filtering은 나와 비슷한 사용자가 사용 / 경험한 특정 제품 또는 서비스를 얼마나 좋아할 지 예측하는 데 사용할 수 있다.

처음부터 Collaborative Filtering을 구현하지는 않을 것이다. 대신 RMSE를 최소화시켜주고 훌륭한 추천을 주는 **SVD**와 같은 강력한 알고리즘을 사용한 **Surprise** library를 사용할 것이다.

In [145]:
reader = Reader()

In [147]:
rating = pd.read_csv('/content/drive/MyDrive/the-movies-dataset/ratings_small.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [150]:
#load_from_df - surprise를 사용하기 위한 형태로 로드
data = Dataset.load_from_df(rating[['userId', 'movieId', 'rating']], reader)
svd = SVD()
#svd 알고리즘을 10-fold cross-validation으로 평가한 결과
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8853  0.8964  0.8958  0.8828  0.8967  0.8846  0.8878  0.9003  0.8988  0.8935  0.8922  0.0061  
MAE (testset)     0.6823  0.6921  0.6907  0.6812  0.6924  0.6775  0.6838  0.6892  0.6935  0.6852  0.6868  0.0053  
Fit time          4.62    2.41    1.88    1.95    1.93    2.67    2.03    1.48    1.72    1.73    2.24    0.86    
Test time         0.28    0.06    0.07    0.46    0.06    0.12    0.06    0.06    0.06    0.06    0.13    0.13    


{'test_rmse': array([0.88534993, 0.89643738, 0.89579712, 0.88279854, 0.89671063,
        0.88462182, 0.88781923, 0.90034489, 0.89884797, 0.8934868 ]),
 'test_mae': array([0.6823212 , 0.69213806, 0.6907252 , 0.68122304, 0.69241034,
        0.67750309, 0.68377098, 0.68917223, 0.6935403 , 0.68523135]),
 'fit_time': (4.6183576583862305,
  2.414240598678589,
  1.882573127746582,
  1.952139139175415,
  1.9269721508026123,
  2.674596071243286,
  2.0346009731292725,
  1.4843924045562744,
  1.7221603393554688,
  1.7298777103424072),
 'test_time': (0.2807176113128662,
  0.0571744441986084,
  0.07275962829589844,
  0.45693469047546387,
  0.06449413299560547,
  0.12021875381469727,
  0.058756113052368164,
  0.06206464767456055,
  0.05554938316345215,
  0.055391788482666016)}

In [152]:
#build_full_trainset - surprise 사용
trainset = data.build_full_trainset()
svd.fit(trainset)

유저를 골라서 그가 준 rating을 확인하자.

In [154]:
rating[rating['userId'] ==1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [155]:
#1번 유저가 302번 영화에 대한 평가 예측: 2.69
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.6856743195771293, details={'was_impossible': False})

# Hybrid Recommender

이전에 사용한 Content Based와 Collabarating filter engine을 이용해 간단한 Hybrid Recommender를 만들 것이다.

*input: user id와 영화 제목

*output: 특정 유저의 예상 등급을 기준으로 유사한 영화의 정렬

In [156]:
def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan

In [160]:
id_map = pd.read_csv('/content/drive/MyDrive/the-movies-dataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
id_map

,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Grumpier Old Men,3,15602.0
Waiting to Exhale,4,31357.0
Father of the Bride Part II,5,11862.0
...,...,...
The Last Brickmaker in America,161944,159550.0
Rustom,162542,392572.0
Mohenjo Daro,162672,402672.0


In [166]:
indices_map = id_map.set_index('id')

In [167]:
indices_map

,movieId
id,
862.0,1
8844.0,2
15602.0,3
31357.0,4
11862.0,5
...,...
159550.0,161944
392572.0,162542
402672.0,162672


In [168]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']

    # content based
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    # collaborative
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [169]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
8712,Guardians of the Galaxy,10014.0,7.9,2014,118340,3.036577
1011,The Terminator,4208.0,7.4,1984,218,3.012281
974,Aliens,3282.0,7.7,1986,679,3.012068
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.985746
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.975652
987,Alien,4564.0,7.9,1979,348,2.937346
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,2.908028
1376,Titanic,7770.0,7.5,1997,597,2.890306
922,The Abyss,822.0,7.1,1989,2756,2.852931
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.779247


In [170]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
974,Aliens,3282.0,7.7,1986,679,3.482961
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.377028
344,True Lies,1138.0,6.8,1994,36955,3.359014
8419,Man of Steel,6462.0,6.5,2013,49521,3.232685
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.216422
987,Alien,4564.0,7.9,1979,348,3.202915
922,The Abyss,822.0,7.1,1989,2756,3.159105
7581,Clash of the Titans,2280.0,5.6,2010,18823,3.110155
1011,The Terminator,4208.0,7.4,1984,218,3.079214
4347,Piranha Part Two: The Spawning,41.0,3.9,1981,31646,3.031426


이 모델은 같은 영화라도 사람마다 다른 영화를 추천하는 것을 알 수 있다. 이는 더 개인적인 유저에게 잘 맞는다.

### 결론
이 노트북에서는 4가지의 다른 아이디어와 알고리즘을 기반으로 한 추천 engine을 만들었다. 다음과 같다 :
1. **Simple Recommender** : 이 시스템은 일반적이고, 특별한 장르의 Top Movie Chart를 위해 전반적인 TMDB 투표수와 투표 평균을 이용했다. IMDB 가중치 평가 시스템은 최종적으로 수행된 평가를 계산하는 데 사용되었다.

2. **Content Based Recommender** : 2가지의 content based engine을 개발하였다. 하나는 예측에 영화 overview와 tagline들을 input으로 사용하였고, 다른 것은 metadata로 cast, crew, genre, keyword를 사용하였다. 또한 더 많은 투표와 더 높은 등급의 영화를 더 선호하도록 간단한 필터를 장치했다.

3. **Collaborative Filtering** : 강력한 Surprise Library를 사용하여 SVD를 기반으로 한 collaborative filter를 개발하였다. RMSE는 1을 넘지 않았고, engine은 주어진 유저와 영화에 대한 추정 등급을 주었다.

4. **Hybrid Engine** : content와 collaborative filtering의 아이디어를 모아 유저에 대해 내부적으로 계산한 추정 등급을 기반으로 특정 사용자에게 영화 추천을 제공하는 엔진을 구축하였다.